**Mosaics the RCM image decompositions and other features from both imaging dates into one**

In [7]:
import os
import rasterio
from rasterio.merge import merge
from rasterio.enums import Resampling

# Set paths for your input folders, output folder, and study area shapefile
folder_path_1 = '2023-05-17 Imagery' #This imagery will take precedence and be on the top of the mosaic
folder_path_2 = '2023-06-02 Imagery'
output_folder = 'RCM Outputs'

# List all files in the folders
files_folder_1 = [os.path.join(folder_path_1, file) for file in os.listdir(folder_path_1) if file.endswith('.tif')]
files_folder_2 = [os.path.join(folder_path_2, file) for file in os.listdir(folder_path_2) if file.endswith('.tif')]

# Extract common identifiers for matching components
common_identifiers = set([os.path.splitext(os.path.basename(file))[0] for file in files_folder_1]) & set([os.path.splitext(os.path.basename(file))[0] for file in files_folder_2])

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Iterate through each common identifier
for identifier in common_identifiers:
    print(f"Processing identifier: {identifier}")

    # Create lists of matching files for each folder
    matching_files_1 = [file for file in files_folder_1 if identifier in file]
    matching_files_2 = [file for file in files_folder_2 if identifier in file]

    # Check if there are matching files in both folders
    if not matching_files_1 or not matching_files_2:
        print(f"Skipping identifier {identifier} - No matching files in one of the folders.")
        continue

    print(f"Matching files in folder 1: {matching_files_1}")
    print(f"Matching files in folder 2: {matching_files_2}")

    # Iterate through each matching pair of components
    for component_path_1, component_path_2 in zip(matching_files_1, matching_files_2):
        print(f"Processing files: {os.path.basename(component_path_1)} and {os.path.basename(component_path_2)}")

        # Read the raster data from both folders
        with rasterio.open(component_path_1) as src1, rasterio.open(component_path_2) as src2:
            # Update metadata for the mosaic
            mosaic_meta = src2.meta.copy()  # Use metadata from folder path 2

            # Mosaic the images
            mosaic, transform = merge([src1, src2], resampling=Resampling.nearest)

            # Update metadata with new mosaic parameters
            mosaic_meta.update({"transform": transform, "height": mosaic.shape[1], "width": mosaic.shape[2]})

            # Set the output path for the mosaicked component
            output_path = os.path.join(output_folder, f"{identifier}.tif")

            # Write the mosaic to a new file
            with rasterio.open(output_path, 'w', **mosaic_meta) as dst:
                dst.write(mosaic)

            print(f"Mosaicked image saved to: {output_path} \n")

print("\n DONE")


Processing identifier: s2psi_orf
Matching files in folder 1: ['2023-05-17 Imagery/s2psi_orf.tif']
Matching files in folder 2: ['2023-06-02 Imagery/s2psi_orf.tif']
Processing files: s2psi_orf.tif and s2psi_orf.tif
Mosaicked image saved to: RCM Outputs/s2psi_orf.tif 

Processing identifier: vol_orf
Matching files in folder 1: ['2023-05-17 Imagery/vol_orf.tif']
Matching files in folder 2: ['2023-06-02 Imagery/vol_orf.tif']
Processing files: vol_orf.tif and vol_orf.tif
Mosaicked image saved to: RCM Outputs/vol_orf.tif 

Processing identifier: s2_orf
Matching files in folder 1: ['2023-05-17 Imagery/s2_orf.tif']
Matching files in folder 2: ['2023-06-02 Imagery/s2_orf.tif']
Processing files: s2_orf.tif and s2_orf.tif
Mosaicked image saved to: RCM Outputs/s2_orf.tif 

Processing identifier: s1_orf
Matching files in folder 1: ['2023-05-17 Imagery/s1_orf.tif']
Matching files in folder 2: ['2023-06-02 Imagery/s1_orf.tif']
Processing files: s1_orf.tif and s1_orf.tif
Mosaicked image saved to: RCM O

In [8]:
import geopandas as gpd
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np

# Specify your folder paths
folder_path = r'RCM Outputs'
shapefile_path = r'Algonquin Park Shapefile/Algonquin_Park_Final.shp'


def reproject_shapefile(shapefile_path, output_epsg=2958):
    # Read the shapefile
    gdf = gpd.read_file(shapefile_path)

    # Reproject to EPSG 2958
    gdf_reprojected = gdf.to_crs(epsg=output_epsg)

    return gdf_reprojected

def clip_rasters(folder_path, shapefile_path):
    # Reproject the shapefile
    reprojected_shapefile = reproject_shapefile(shapefile_path)

    # Loop through each raster file in the folder
    for file in os.listdir(folder_path):
        if file.endswith(".tif"):
            input_path = os.path.join(folder_path, file)

            try:
                # Open the raster file
                with rasterio.open(input_path) as src:
                    # Mask the raster using the reprojected shapefile
                    out_image, out_transform = mask(src, reprojected_shapefile.geometry, crop=True)
                    
                    # Set all values of 255 to null
                    out_image[out_image == 255] = np.nan

                    # Update metadata for the clipped raster
                    out_meta = src.meta.copy()
                    out_meta.update({
                        'driver': 'GTiff',
                        'height': out_image.shape[1],
                        'width': out_image.shape[2],
                        'transform': out_transform
                    })

                    # Overwrite the existing raster file
                    with rasterio.open(input_path, 'w', **out_meta) as dest:
                        dest.write(out_image)

                print(f"Clipping, reprojecting and overwriting {file} completed.")
                
            except Exception as e:
                print(f"Error processing {file}: {e}")

    print("Clipping, reprojecting, and overwriting files completed.")

# Run the script
clip_rasters(folder_path, shapefile_path)


Clipping and overwriting alpha_orf.tif completed.
Clipping and overwriting phi_orf.tif completed.
Clipping and overwriting lpRatio_orf.tif completed.
Clipping and overwriting ml_orf.tif completed.
Clipping and overwriting surf_orf.tif completed.
Clipping and overwriting s2psi_orf.tif completed.
Clipping and overwriting m_orf.tif completed.
Clipping and overwriting vol_orf.tif completed.
Clipping and overwriting s0_orf.tif completed.
Clipping and overwriting dbl_orf.tif completed.
Clipping and overwriting mu_orf.tif completed.
Clipping and overwriting s1_orf.tif completed.
Clipping and overwriting delta_orf.tif completed.
Clipping and overwriting CV_mrsb_cs_c2d_orf.tif completed.
Clipping and overwriting s2chi_orf.tif completed.
Clipping and overwriting s3_orf.tif completed.
Clipping and overwriting CH_mrsb_cs_c2d_orf.tif completed.
Clipping and overwriting cpRatio_orf.tif completed.
Clipping and overwriting s2_orf.tif completed.
Clipping and overwriting mc_orf.tif completed.
Clipping, 